In [8]:
import math

# get user input and handle invalid input
def get_input(prompt, data_type=float):
    while True:
        try:
            return data_type(input(prompt))
        except ValueError:
            print("Invalid input. Please try again.")

# calculate the present value of an annuity
def calculate_pv_annuity(cash_flow, rate, periods, growth_rate):
    return cash_flow * ((1 - ((1 + growth_rate)/(1 + rate)) ** periods) / (rate - growth_rate))

# calculate the present value of a perpetuity
def calculate_pv_perpetuity(cash_flow, discount_rate, growth_rate):
    return cash_flow * (1 + growth_rate) / (discount_rate - growth_rate)

# calculate the present value of a future amount
def calculate_pv(future_value, rate, periods):
    return future_value / (1 + rate) ** periods

# calculate the future value of a present amount
def calculate_fv(present_value, rate, periods):
    return present_value * (1 + rate) ** periods

def main():
    print("Present Value / Future Value Calculator")
    
    # determine calculation type from user (PV or FV)
    calc_type = get_input("Enter 'PV' for Present Value or 'FV' for Future Value: ", str).upper()
    if calc_type not in ['PV', 'FV']:
        print("Invalid calculation type. Exiting.")
        return

    # input values from user
    discrete_periods = get_input("Enter the number of discrete forecast periods: ", int)
    growth_phases = get_input("Enter the number of growth phases (including 'none'): ", int)
    discount_rate = get_input("Enter the applicable discount rate (as a decimal): ")

    total_value = 0
    breakdown = []

    # calculate PV for discrete forecast periods
    if calc_type == 'PV':
        for i in range(discrete_periods):
            cash_flow = get_input(f"Enter cash flow for discrete period {i+1}: ")
            pv = calculate_pv(cash_flow, discount_rate, i+1)
            total_value += pv
            breakdown.append(f"PV Discrete Period {i+1}: ${pv:.2f}")

    # calculate PV/FV for growth phases
    for phase in range(growth_phases):
        growth_rate = get_input(f"Enter growth rate for phase {phase+1} (as a decimal): ")
        phase_length = get_input(f"Enter length of phase {phase+1} (in periods): ", int)
        
        # determine if the current phase is the last one
        is_last_phase = phase == growth_phases - 1
        if is_last_phase:
            phase_type = get_input("Enter 'A' for annuity or 'P' for perpetuity: ", str).upper()
        else:
            phase_type = 'A'  # default to annuity for intermediate phases

        if phase_type == 'A':
            cash_flow = get_input(f"Enter initial cash flow for phase {phase+1}: ")
            if calc_type == 'PV':
                pv_phase = calculate_pv_annuity(cash_flow, discount_rate, phase_length, growth_rate)
                pv_phase = calculate_pv(pv_phase, discount_rate, sum(breakdown[p][1] for p in range(phase) if isinstance(breakdown[p], tuple)))
                total_value += pv_phase
                breakdown.append((f"PV Phase {phase+1} Annuity", phase_length, f"${pv_phase:.2f}"))
            else:  # FV
                fv_phase = cash_flow * ((1 + growth_rate) ** phase_length - 1) / growth_rate
                total_value += fv_phase
                breakdown.append((f"FV Phase {phase+1} Annuity", phase_length, f"${fv_phase:.2f}"))
        elif phase_type == 'P':
            cash_flow = get_input(f"Enter initial cash flow for perpetuity: ")
            if calc_type == 'PV':
                pv_perpetuity = calculate_pv_perpetuity(cash_flow, discount_rate, growth_rate)
                pv_perpetuity = calculate_pv(pv_perpetuity, discount_rate, sum(breakdown[p][1] for p in range(phase) if isinstance(breakdown[p], tuple)))
                total_value += pv_perpetuity
                breakdown.append((f"PV Phase {phase+1} Perpetuity", math.inf, f"${pv_perpetuity:.2f}"))
            else:  # FV
                print("Future Value calculation for perpetuity is not applicable.")

    # print the breakdown of calculations and the total value
    print("\nBreakdown:")
    for item in breakdown:
        if isinstance(item, tuple):
            print(f"{item[0]}: {item[2]}")
        else:
            print(item)
    
    print(f"\nTotal {'PV' if calc_type == 'PV' else 'FV'} = ${total_value:.2f}")

if __name__ == "__main__":
    main()

Present Value / Future Value Calculator

Breakdown:
PV Phase 1 Annuity: $4384.42
PV Phase 2 Perpetuity: $7570.33

Total PV = $11954.75
